# 네이버 블로그 크롤링 by selenium - 제주맛집

In [3]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import quote 
import time

In [2]:
# 검색어에 '제주맛집' 집어넣어야 해서 [from urllib.parse import quote ] 추가한다고 함

In [4]:
url = f'https://search.naver.com/search.naver?where=view&sm=tab_jum&query={quote("제주맛집")}'
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)
time.sleep(2)

In [5]:
# Blog 페이지를 아래로 내려서 많은 데이터 확보
from selenium.webdriver.common.keys import Keys

body = driver.find_element_by_tag_name('body')
for _ in range(30):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)

- 제목, 컨텐츠, 날짜 가져오기 첫 페이지에서만

In [6]:
lis = driver.find_elements_by_css_selector('li.bx._svp_item')
len(lis)

30

In [7]:
li = lis[0]
title = li.find_element_by_css_selector('a.api_txt_lines.total_tit._cross_trigger').text().strip()
title

TypeError: 'str' object is not callable

In [7]:
date = li.select_one('.sub_time.sub_txt').text().strip()
date

'2021.11.19.'

In [8]:
# But 그 다음 데이터는 정확한 날짜가 아니라..<<보류 
lis[1].select_one('.sub_time.sub_txt').text().strip()

'6일 전'

In [9]:
content = li.select_one('.api_txt_lines.dsc_txt').text().strip()
content

'지난 주말 친구들과 제주여행을 떠났다가 이름알려진 제주 맛집에 갔다왔어요. 전복뚝배기부터 고등어조림,쌈밥 등 풍성한 음식에 아름다운 바다풍경까지 감상할 수 있었던 식당으로 만족스러웠던 곳이랍니다. 1....'

In [10]:
href = li.select_one('a.api_txt_lines.total_tit._cross_trigger')['href']
href

'https://blog.naver.com/tok4/222573156818'

- 날짜 처리

In [11]:
from datetime import datetime, timedelta

today = datetime.now() # datetime.today() 
today

datetime.datetime(2021, 12, 8, 11, 29, 52, 918118)

In [12]:
today.year, today.month, today.day

(2021, 12, 8)

In [13]:
# 어제 
date = today - timedelta(1)
f'{date.year}.{date.month:02d}.{date.day:02d}.'



'2021.12.07.'

In [14]:
# 6일 전 
date = today - timedelta(6)
f'{date.year}.{date.month:02d}.{date.day:02d}.'

'2021.12.02.'

In [15]:
# 13시간 전 
date = today - timedelta(hours=13)
f'{date.year}.{date.month:02d}.{date.day:02d}.'

'2021.12.07.'

In [16]:
# 45분 전 
date = today - timedelta(minutes=45)
f'{date.year}.{date.month:02d}.{date.day:02d}.'

'2021.12.08.'

In [32]:
def convert_date(s): #s 라는 str

    if len(s) == 11:
        return s[:-1]  # 끝 .없애기

    today = datetime.now() # datetime.today() 

    if s == '어제':
        date = today - timedelta(1)

    elif '일' in s:
        date = today - timedelta(int(s[:-3]))
        
    elif '시간' in s:
        date = today - timedelta(hours=int(s[:-4]))
        
    elif '분' in s:  
        date = today - timedelta(minutes=int(s[:-3]))

    return f'{date.year}.{date.month:02d}.{date.day:02d}'      
             

In [33]:
convert_date('6일 전')


'2021.12.02'

In [34]:
convert_date('3일 전')

'2021.12.05'

In [35]:
convert_date('12시간 전')

'2021.12.07'

In [29]:
convert_date('15분 전')

'2021.12.08.'

In [36]:
lines = []
for li in lis:
    title = li.select_one('a.api_txt_lines.total_tit._cross_trigger').get_text().strip()
    date = li.select_one('.sub_time.sub_txt').get_text().strip()
    content = li.select_one('.api_txt_lines.dsc_txt').get_text().strip()
    href = li.select_one('a.api_txt_lines.total_tit._cross_trigger')['href']
    lines.append([title, convert_date(date), content, href])
    
    
    

In [41]:
df = pd.DataFrame(lines, columns=['제목', '날짜', '내용', 'href'])
df[['제목','날짜','내용']].head()

,제목,날짜,내용
0,만족뿐인 제주 맛집 공유,2021.11.19,지난 주말 친구들과 제주여행을 떠났다가 이름알려진 제주 맛집에 갔다왔어요. 전복뚝배...
1,또 생각났던 제주 맛집,2021.12.02,제가 시킨 제주 비빔국수는 갖은 야채들이 그릇에 알록달록 예쁘게 가득 채워져 있었는...
2,북적였던 제주 맛집 리스트,2021.11.13,무엇보다 비린내 한번 느껴지지 않아 배가 부른 상태였음에도 쉽사리 남길 수가 없었던...
3,감격했던 제주 맛집 소개,2021.11.15,얼마 전 친구들과 제주로 여행갔다가 현지인께 추천받은 제주 맛집에 다녀왔어요. 갓 ...
4,제주도 가볼만한곳 제주 맛집,2021.12.01,제주도 가볼만한곳 제주 맛집은 깔끔한 메뉴가 푸짐하게 나오기로 소문 난 곳이라 부담...


In [42]:
df.to_csv('네이버블로그.csv')

In [1]:
driver.close()

NameError: name 'driver' is not defined